# Artifact rejection / repair with linear regression

## important channels

- EOG002
- BIO004 is ECG (ECG003 not working proper when data was recorded)
- BIO005 is data of Breathfrequency
- EEG001-EEG030 (EEG electrodes / channels 1-30)
- MEG0111-MEG2642 (MEG electrodes / channels 111-2642)

## initialize data structure

In [1]:
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import EOGRegression
%matplotlib qt

In [2]:
# Load seperated data files
raw = mne.io.read_raw_fif('SSP_eog_applied_raw.fif', preload=True)

Opening raw data file SSP_eog_applied_raw.fif...
    Read a total of 3 projection items:
        EOG-planar--0.200-0.200-PCA-01 (1 x 204) active
        EOG-axial--0.200-0.200-PCA-01 (1 x 102) active
        EOG-eeg--0.200-0.200-PCA-01 (1 x 30) active
    Range : 25000 ... 1583999 =     50.000 ...  3167.998 secs
Ready.
Opening raw data file /home/ijekt/Documents/cs_cog_fat/SSP_eog_applied_raw-1.fif...
    Read a total of 3 projection items:
        EOG-planar--0.200-0.200-PCA-01 (1 x 204) active
        EOG-axial--0.200-0.200-PCA-01 (1 x 102) active
        EOG-eeg--0.200-0.200-PCA-01 (1 x 30) active
    Range : 1584000 ... 2135999 =   3168.000 ...  4271.998 secs
Ready.
Reading 0 ... 2110999  =      0.000 ...  4221.998 secs...


In [3]:
raw.plot()

Using qt as 2D backend.


Channels marked as bad:
none


In [3]:
print(raw.info)

<Info | 16 non-empty values
 acq_pars: ACQactiveGround 0 ACQch.BIO001.gain 2000 ACQch.BIO001.highpass ...
 bads: []
 ch_names: EOG002, ECG003, BIO004, BIO005, EEG001, EEG002, EEG003, EEG004, ...
 chs: 1 EOG, 1 ECG, 2 BIO, 30 EEG, 102 Magnetometers, 204 Gradiometers, 2 Stimulus, 1 SYST
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 dig: 544 items (3 Cardinal, 5 HPI, 31 EEG, 505 Extra)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 200.0 Hz
 meas_date: 2023-09-01 12:50:17 UTC
 meas_id: 4 items (dict)
 nchan: 343
 proc_history: 1 item (list)
 projs: []
 sfreq: 500.0 Hz
 subject_info: 6 items (dict)
>


In [4]:
raw.rename_channels({'BIO004' : 'ECG'})

<Raw | cogfat_rest_18_ok24-rall_mci_ds2_raw_sss.fif, 343 x 2111000 (4222.0 s), ~5.40 GB, data loaded>

In [5]:
channel_type_mapping = {'ECG003': 'bio'}  # Map your renamed channel to the appropriate channel type
raw.set_channel_types(channel_type_mapping)

<Raw | cogfat_rest_18_ok24-rall_mci_ds2_raw_sss.fif, 343 x 2111000 (4222.0 s), ~5.40 GB, data loaded>

In [6]:
channel_type_mapping = {'ECG': 'ecg'}  # Map your renamed channel to the appropriate channel type
raw.set_channel_types(channel_type_mapping)

<Raw | cogfat_rest_18_ok24-rall_mci_ds2_raw_sss.fif, 343 x 2111000 (4222.0 s), ~5.40 GB, data loaded>

In [7]:
# Get the indices of EEG and MEG channels
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=False, ecg=False, stim=False, bio=False, syst=False)


# Apply a bandpass filter to MEG and EEG data (1 Hz to 200 Hz)
raw.filter(l_freq=1, h_freq=200, picks=picks)

# Save the filtered data
raw.save('pre_filtered_data_raw.fif', overwrite=True)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 200.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 225.00 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    3.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    8.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   19.1s


Writing /home/ijekt/Documents/cog_fat_study/pre_filtered_data_raw.fif
Closing /home/ijekt/Documents/cog_fat_study/pre_filtered_data_raw.fif
Writing /home/ijekt/Documents/cog_fat_study/pre_filtered_data_raw-1.fif
Closing /home/ijekt/Documents/cog_fat_study/pre_filtered_data_raw-1.fif
[done]


In [4]:
raw = mne.io.read_raw_fif('pre_filtered_data_raw.fif', preload=True)

Opening raw data file pre_filtered_data_raw.fif...
    Range : 25000 ... 1583999 =     50.000 ...  3167.998 secs
Ready.
Opening raw data file /home/ijekt/Documents/cs_cog_fat/pre_filtered_data_raw-1.fif...
    Range : 1584000 ... 2135999 =   3168.000 ...  4271.998 secs
Ready.
Reading 0 ... 2110999  =      0.000 ...  4221.998 secs...


In [ ]:
raw.compute_psd(fmax=200).plot(picks="data", exclude="bads")
raw.plot(duration=5, n_channels=20, scalings='auto')

# LOGFILE related

In [18]:
log = pd.read_csv('Results_CFRm_TDB_018_trial.txt', delimiter='\t')
log.head()

,Time,Subject,ID,Age,Sex,Handedness,Condition,Type,Letter,TargetLetter,...,RTPDigit,RCDigit,HITDigit,MISSDigit,FADigit,CORDigit,KDigit,Block,Trial,STD
0,20239115231,ok24,18,28,1,2,LL,TDB,N,0,...,0.924,1,1,0,0,0,56,1,1,0.999
1,20239115231,ok24,18,28,1,2,LL,TDB,N,1,...,0.694,1,0,0,0,1,55,1,2,0.999
2,20239115231,ok24,18,28,1,2,LL,TDB,L,0,...,0.818,1,1,0,0,0,56,1,3,0.999
3,20239115231,ok24,18,28,1,2,LL,TDB,A,0,...,0.559,1,0,0,0,1,55,1,4,0.999
4,20239115231,ok24,18,28,1,2,LL,TDB,N,0,...,0.776,1,0,0,0,1,55,1,5,0.999


In [ ]:
# Trigger offset
triggeroffset_letter = -0.03 * raw.info['sfreq']
triggeroffset_digit = -0.03 * raw.info['sfreq']

# Trial durations
Ttrial_s = [-0.2, 1.5]
Ttrial_r = [-0.5, 0.8]

ntr = len(log)
epoch[iisub]['trialdur_s'] = np.arange(round(Ttrial_s[0] * raw.info['sfreq']),
                                      round(Ttrial_s[1] * raw.info['sfreq']) + 1)
epoch[iisub]['lepoch_s'] = len(epoch[iisub]['trialdur_s'])

epoch[iisub]['trialdur_r'] = np.arange(round(Ttrial_r[0] * raw.info['sfreq']),
                                      round(Ttrial_r[1] * raw.info['sfreq']) + 1)
epoch[iisub]['lepoch_r'] = len(epoch[iisub]['trialdur_r'])

# Trial starts
trialstart_sletter = logtbl['ti_letter'] + triggeroffset_letter
epoch[iisub]['trials_sletter'] = trialstart_sletter[:, np.newaxis] + epoch[iisub]['trialdur_s']

rt_letter = np.nanmedian(logtbl['RTLetter'])
rt_letter[np.isnan(rt_letter)] = np.nanmedian(rt_letter)
trialstart_rletter = trialstart_sletter + np.floor(raw.info['sfreq'] * rt_letter)
epoch[iisub]['trials_rletter'] = trialstart_rletter[:, np.newaxis] + epoch[iisub]['trialdur_r']

trialstart_sdigit = logtbl['ti_digit'] + triggeroffset_digit
epoch[iisub]['trials_sdigit'] = trialstart_sdigit[:, np.newaxis] + epoch[iisub]['trialdur_s']

rt_digit = np.nanmedian(logtbl['RTDigit'])
rt_digit[np.isnan(rt_digit)] = np.nanmedian(rt_digit)
trialstart_rdigit = trialstart_sdigit + np.floor(raw.info['sfreq'] * rt_digit)
epoch[iisub]['trials_rdigit'] = trialstart_rdigit[:, np.newaxis] + epoch[iisub]['trialdur_r']

# Trial types
epoch[iisub]['trialtype_letter'] = np.zeros(ntr)
epoch[iisub]['trialtype_letter'][logtbl['HITLetter'] == 1] = 1
epoch[iisub]['trialtype_letter'][logtbl['MISSLetter'] == 1] = 2
epoch[iisub]['trialtype_letter'][logtbl['FALetter'] == 1] = 3
epoch[iisub]['trialtype_letter'][logtbl['CORLetter'] == 1] = 4

epoch[iisub]['trialtype_digit'] = np.zeros(ntr)
epoch[iisub]['trialtype_digit'][logtbl['HITDigit'] == 1] = 1
epoch[iisub]['trialtype_digit'][logtbl['MISSDigit'] == 1] = 2
epoch[iisub]['trialtype_digit'][logtbl['FADigit'] == 1] = 3
epoch[iisub]['trialtype_digit'][logtbl['CORDigit'] == 1] = 4

# Sample rate and time
sr = raw.info['sfreq']
epoch[iisub]['time_s'] = epoch[iisub]['trialdur_s'] / sr
epoch[iisub]['time_r'] = epoch[iisub]['trialdur_r'] / sr

# set epochs

In [17]:
del raw